In [ ]:
## User Registration

import os
import cv2
import face_recognition
import numpy as np
import faiss

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}

def register_user_with_webcam(empid, name):
    # Create a folder for the user using empid
    user_folder = os.path.join("users", empid)
    if not os.path.exists(user_folder):
        os.makedirs(user_folder)

    # Capture image from the webcam
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Register")

    print("Press SPACE to capture the image and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Register", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            img_name = os.path.join(user_folder, f"{empid}.jpg")
            cv2.imwrite(img_name, frame)
            print(f"{img_name} saved!")
            break

    cam.release()
    cv2.destroyAllWindows()

    # Load the image and convert it to RGB
    image = cv2.imread(img_name)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face and encode
    face_locations = face_recognition.face_locations(rgb_image)
    face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

    if len(face_encodings) > 0:
        face_encoding = face_encodings[0]
        # Store in FAISS
        index.add(np.array([face_encoding]))
        # Map empid to index and store user info
        user_db[empid] = {
            "index": index.ntotal - 1,
            "name": name
        }
        print(f"User {name} (empid: {empid}) registered successfully.")
    else:
        print("No face found in the image.")

# Example usage
empid = input("Enter your employee ID: ")
name = input("Enter your name: ")
register_user_with_webcam(empid, name)


In [3]:
import os
import cv2
import face_recognition
import numpy as np
import faiss

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}  # Ideally, load this from a persistent storage like a database

def load_user_database():
    # Simulated function to load user database from a persistent storage
    registered_users_folder = "registered"
    if not os.path.exists(registered_users_folder):
        print("No registered users found.")
        return

    for empid in os.listdir(registered_users_folder):
        user_folder = os.path.join(registered_users_folder, empid)
        if os.path.isdir(user_folder):
            image_path = os.path.join(user_folder, f"{empid}.jpg")
            if os.path.exists(image_path):
                # Load the image and convert it to RGB
                image = cv2.imread(image_path)
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Detect face and encode
                face_locations = face_recognition.face_locations(rgb_image)
                face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

                if len(face_encodings) > 0:
                    face_encoding = face_encodings[0]

                    # Store in FAISS
                    index.add(np.array([face_encoding]))

                    # Map empid to index and store user info
                    user_db[empid] = {
                        "index": index.ntotal - 1,
                        "folder": user_folder,  # Store user folder path for later verification
                        "image_path": image_path  # Store path to image for deletion later
                    }
                    print(f"Loaded user {empid} from {user_folder}.")
                else:
                    print(f"No face found in the image {image_path}.")

def verify_user_with_webcam():
    load_user_database()  # Load user data into user_db

    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Verify")

    user_info = None  # Initialize user_info variable

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Show the frame to the user
        cv2.imshow("Verify", frame)

        # Load the image and convert it to RGB
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face and encode
        face_locations = face_recognition.face_locations(rgb_image)
        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

        if len(face_encodings) > 0:
            face_encoding = face_encodings[0]

            # Search in FAISS index
            query = np.array([face_encoding])
            D, I = index.search(query, 1)

            if D[0][0] < 0.6:  # Adjust this threshold as needed
                # Match found
                empid = list(user_db.keys())[I[0][0]]
                user_info = user_db[empid]
                print(f"Authorized: User {empid}, Folder: {user_info['folder']}")
                break
            else:
                print("Not Authorized: Face doesn't match any registered user.")
                break
        else:
            print("No face found in the image for verification.")

        # Close the window after a short delay (100ms)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    # Release the camera and close all windows
    cam.release()
    cv2.destroyAllWindows()

    # Delete the captured image (if any)
    if user_info and 'image_path' in user_info:
        try:
            os.remove(user_info['image_path'])
            print(f"Deleted temporary image: {user_info['image_path']}")
        except Exception as e:
            print(f"Failed to delete temporary image: {user_info['image_path']}. Error: {e}")

# Example usage for verification
if __name__ == "__main__":
    verify_user_with_webcam()


No registered users found.
Not Authorized: Face doesn't match any registered user.


In [7]:
import os
import cv2
import face_recognition
import numpy as np
import faiss

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}  # Ideally, load this from a persistent storage like a database

def load_user_database():
    # Simulated function to load user database from a persistent storage
    registered_users_folder = "registered"
    if not os.path.exists(registered_users_folder):
        print("No registered users found.")
        return

    for empid in os.listdir(registered_users_folder):
        user_folder = os.path.join(registered_users_folder, empid)
        if os.path.isdir(user_folder):
            image_path = os.path.join(user_folder, f"{empid}.jpg")
            if os.path.exists(image_path):
                # Load the image and convert it to RGB
                image = cv2.imread(image_path)
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Detect face and encode
                face_locations = face_recognition.face_locations(rgb_image)
                face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

                if len(face_encodings) > 0:
                    face_encoding = face_encodings[0]

                    # Store in FAISS
                    index.add(np.array([face_encoding]))

                    # Map empid to index and store user info
                    user_db[empid] = {
                        "index": index.ntotal - 1,
                        "folder": user_folder,  # Store user folder path for later verification
                        "image_path": image_path  # Store path to image for deletion later
                    }
                    print(f"Loaded user {empid} from {user_folder}.")
                else:
                    print(f"No face found in the image {image_path}.")

def print_all_users():
    if not user_db:
        print("No users loaded in the database.")
        return

    print("All users in the database:")
    for empid, user_info in user_db.items():
        print(f"Employee ID: {empid}")
        print(f"User Folder: {user_info['folder']}")
        print(f"Image Path: {user_info['image_path']}")
        print(f"Index in FAISS: {user_info['index']}")
        print("")

def verify_user_with_webcam():
    load_user_database()  # Load user data into user_db
    print_all_users()     # Print all user data after loading

    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Verify")

    user_info = None  # Initialize user_info variable

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Show the frame to the user
        cv2.imshow("Verify", frame)

        # Load the image and convert it to RGB
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face and encode
        face_locations = face_recognition.face_locations(rgb_image)
        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

        if len(face_encodings) > 0:
            face_encoding = face_encodings[0]

            # Search in FAISS index
            query = np.array([face_encoding])
            D, I = index.search(query, 1)

            if D[0][0] < 0.6:  # Adjust this threshold as needed
                # Match found
                empid = list(user_db.keys())[I[0][0]]
                user_info = user_db[empid]
                print(f"Authorized: User {empid}, Folder: {user_info['folder']}")
                break
            else:
                print("Not Authorized: Face doesn't match any registered user.")
                break
        else:
            print("No face found in the image for verification.")

        # Close the window after a short delay (100ms)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    # Release the camera and close all windows
    cam.release()
    cv2.destroyAllWindows()

    # Delete the captured image (if any)
    if user_info and 'image_path' in user_info:
        try:
            os.remove(user_info['image_path'])
            print(f"Deleted temporary image: {user_info['image_path']}")
        except Exception as e:
            print(f"Failed to delete temporary image: {user_info['image_path']}. Error: {e}")

# Example usage for verification
if __name__ == "__main__":
    verify_user_with_webcam()


No registered users found.
No users loaded in the database.
Not Authorized: Face doesn't match any registered user.
